In [2]:
import os

def read_experiment_data(folder_path):
    # Desired order of experiments
    experiment_order = [
        'direct', 'exp6', 'exp61', 'exp76', 'exp4.4', 'exp53', 'exp52'
    ]

    # Desired order of LMs
    lm_order = ['phi3', 'aya', 'command-r']

    # Initialize experiment_data with all values set to "-1±-1\%"
    experiment_data = {exp: {lm: "-1±-1\%" for lm in lm_order} for exp in experiment_order}

    # Mapping for experiment names
    experiment_name_mapping = {
        'direct': 'direct asking',
        'exp6': 'CoT',
        'exp61': 'PS prompting',
        'exp4.4': '1 inst. 2 messages',
        'exp52': '2 inst.',
        'exp76': '1 inst. 1 message',
        'exp53': '1 inst. info. copied'
    }

    # Read each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Extract experiment name and language model name from the first line
                first_line_parts = lines[0].strip().split()
                experiment_name = next((part.replace(":", "") for part in first_line_parts if part.startswith('exp') or part == 'direct'), None)
                language_model = first_line_parts[-1].split(':')[0]
                
                # Extract accuracy and SD from the file
                accuracy = None
                sd = None
                for line in lines:
                    if line.startswith('accuracy:'):
                        accuracy = float(line.split(':')[1].strip())
                    if line.startswith('SD:'):
                        sd = float(line.split(':')[1].strip())
                
                # Format accuracy and SD
                if accuracy is not None and sd is not None and experiment_name in experiment_data and language_model in lm_order:
                    accuracy_percent = f"{accuracy * 100:.2f}".rstrip('0').rstrip('.')
                    sd_percent = f"{sd * 100:.1f}".rstrip('0').rstrip('.') + "\\%"
                    result = f"{accuracy_percent}±{sd_percent}"
                
                    # Store the formatted result in the dictionary
                    experiment_data[experiment_name][language_model] = result

    # Calculate averages for each experiment
    experiment_averages = {
        exp: sum(float(experiment_data[exp][lm].split('±')[0]) for lm in lm_order) / len(lm_order)
        for exp in experiment_data
    }

    # Get the average for "CoT"
    CoT_average = experiment_averages.get('exp6', None)

    # Generate LaTeX table
    table_header = "Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\\\"

    table_rows = []
    for exp in experiment_order:
        if exp in experiment_data:
            mapped_experiment_name = experiment_name_mapping.get(exp, exp)
            row = mapped_experiment_name
            for lm in lm_order:
                value = experiment_data[exp].get(lm)
                accuracy_value = float(value.split('±')[0])
                
                # Check if value is the highest in its column
                if accuracy_value == max(float(experiment_data[exp][lm].split('±')[0]) for exp in experiment_order if lm in experiment_data[exp]):
                    row += (" & \\textbf" + '{' + f"{value}" + "}")
                else:
                    row += f" & {value}"

            # Calculate and format the average difference
            avg_diff = experiment_averages[exp] - CoT_average
            if avg_diff > 0:
                avg_diff_str = f"\\textcolor{{blue}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')
            else:
                avg_diff_str = f"\\textcolor{{red}}{{{avg_diff:.2f}}}".rstrip('0').rstrip('.')

            row += f" & {avg_diff_str}"
            table_rows.append(row)

    table_body = " \\\\\n".join(table_rows)
    latex_table = (
        "\\begin{table}[h!]\n"
        "\\caption{StrategyQA}\n"
        "\\begin{tabular}{l" + "c" * (len(lm_order) + 1) + "}\n"
        "\\hline\n"
        + table_header + "\n\\hline\n"
        + table_body + " \\\\" + "\n\\hline\n"
        "\\end{tabular}\n"
        "\\end{table}"
    )

    return latex_table


# Example usage
folder_path = '/home/jyu48/PREP/StrategyQA/i'
latex_code = read_experiment_data(folder_path)
print(latex_code)


\begin{table}[h!]
\caption{StrategyQA}
\begin{tabular}{lcccc}
\hline
Experiment & Phi-3 & Aya & Command-R & Avg. Diff \\
\hline
direct asking & 63±2.2\% & 76.4±1.9\% & 78.8±1.8\% & \textcolor{blue}{0.67} \\
CoT & 59.2±2.2\% & 78.6±1.8\% & 78.4±1.8\% & \textcolor{red}{0.00} \\
PS prompting & 67±2.1\% & 77.2±1.9\% & 80.2±1.8\% & \textcolor{blue}{2.73} \\
1 inst. 1 message & 64.8±2.1\% & \textbf{80.4±1.8\%} & 79±1.8\% & \textcolor{blue}{2.67} \\
1 inst. 2 messages & 66.4±2.1\% & 79.4±1.8\% & 77.8±1.9\% & \textcolor{blue}{2.47} \\
1 inst. info. copied & 65±2.1\% & 80±1.8\% & 78.4±1.8\% & \textcolor{blue}{2.40} \\
2 inst. & \textbf{70.8±2\%} & 77.8±1.9\% & \textbf{80.4±1.8\%} & \textcolor{blue}{4.27} \\
\hline
\end{tabular}
\end{table}


<>:13: SyntaxWarning: invalid escape sequence '\%'
<>:13: SyntaxWarning: invalid escape sequence '\%'
/tmp/ipykernel_101033/2019025098.py:13: SyntaxWarning: invalid escape sequence '\%'
  experiment_data = {exp: {lm: "-1±-1\%" for lm in lm_order} for exp in experiment_order}
